# imports

In [1]:
import pandas as pd

## constants

In [2]:
NUMBER_OF_ELEMENTS_TO_TAKE = 200

# Preprocess dataframe

## Load data and concat two years

In [3]:
data_2018 = pd.read_csv("data2018.csv", encoding="cp1251")
data_2019 = pd.read_csv("data2019.csv", encoding="cp1251")
df = pd.concat([data_2018, data_2019])
df = df.sample(1100000, random_state=2).reset_index(drop=True)
df

,Рік,Область,Учбовий заклад,Спеціальність,Факультет,Місце у рейтингу,Пріоритет,Конкурсний бал,Статус заяви,Інфо про спеціальність,Квоти
0,2019,м. Київ,Київський національний торговельно-економічний...,081 Право,Факультет міжнародної торгівлі та права,754,К,155.900,Допущено,Українська мова та література (ЗНО): 154\r\nІс...,—
1,2018,Львівська область,Львівський національний університет імені Іван...,035 Філологія,Філологічний факультет,390,К,165.852,Допущено,Середній бал документа про освіту: 10.6\r\nУкр...,—
2,2019,Львівська область,Львівський національний університет імені Іван...,"291 Міжнародні відносини, суспільні комунікаці...",Факультет міжнародних відносин,480,4,187.068,Допущено,—,—
3,2018,м. Київ,Київський національний лінгвістичний університет,081 Право,Факультет економіки і права,26,6,193.850,Рекомендовано (контракт),Українська мова та література (ЗНО): 193\r\nІс...,—
4,2019,м. Київ,Національний університет харчових технологій,073 Менеджмент,Навчально-науковий інститут економіки і управл...,162,К,119.400,Рекомендовано (к),Українська мова та література (ЗНО): 131\r\nМа...,—
5,2019,Миколаївська область,Чорноморський національний університет імені П...,"291 Міжнародні відносини, суспільні комунікаці...",Політичних наук,15,—,165.880,Рекомендовано (к),Українська мова та література (ЗНО): 163\r\nІс...,—
6,2018,Запорізька область,Таврійський державний агротехнологічний універ...,"072 Фінанси, банківська справа та страхування",Факультет Економіки та бізнесу,1,—,164.475,До наказу (контракт),Українська мова та література (ЗНО): 174\r\nІс...,—
7,2018,Львівська область,Львівський національний університет імені Іван...,014 Середня освіта,Філологічний факультет,378,5,166.932,Допущено,Українська мова та література (ЗНО): 181\r\nІс...,—
8,2018,м. Київ,Київський національний університет культури і ...,034 Культурологія,Факультет соціокультурної діяльності,340,3,164.300,Відхилено (контракт),Українська мова та література (ЗНО): 170\r\nІс...,—
9,2018,м. Київ,Київський університет імені Бориса Грінченка,035 Філологія,Інститут філології,550,1,167.700,Допущено,Українська мова та література (ЗНО): 178\r\nІн...,—


## Rename, remove and clean columns
0 - didn't pass at all

1 - pass to contract

2 - pass to budget

In [4]:
df.columns = [
    "year",
    "district",
    "university",
    "specialty",
    "faculty",
    "rating",
    "priority",
    "points",
    "status",
    "info",
    "quotas"
]
df["status"] = df["status"].map({
    'До наказу (контракт)': 1,
    'Рекомендовано (контракт)': 1,
    'Відмова': 0,
    'Скасовано НЗ': 0,
    'Скасовано (втрата пріор.)': 0,
    'Зареєстровано': 0,
    'Затримано': 0,
    'Відраховано': 0,
    'До наказу (бюджет)': 2,
    'Відхилено (бюджет)': 0,
    'Допущено': 0,
    'Рекомендовано (бюджет)': 2,
    'Заява надійшла з сайту': 0,
    'Відхилено (контракт)': 0,
    'Допущено (контракт, за ріш. ПК)': 1,
    'Рекомендовано (к)': 1,
    'До наказу (к)': 1,
    'До наказу (б)': 2,
    'Відхилено (б)': 0,
    'Рекомендовано (б)': 2,
    'Відхилено (к)': 0,
    'Допущено (к, за ріш. ПК)': 1
})
df["specialty"] = df["specialty"].map(lambda val: val[:3])

In [5]:
df = df.query("points <= 200 and points >= 100")
districts = df.district.unique()
specialties = df.specialty.unique()
accum = []
for dis in districts:
    accum.append(df.loc[(df["district"] == dis) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["district"] == dis) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["district"] == dis) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
for spec in specialties:
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
df = pd.concat(accum)
df

,year,district,university,specialty,faculty,rating,priority,points,status,info,quotas
0,2019,м. Київ,Київський національний торговельно-економічний...,081,Факультет міжнародної торгівлі та права,754,К,155.900,0,Українська мова та література (ЗНО): 154\r\nІс...,—
8,2018,м. Київ,Київський національний університет культури і ...,034,Факультет соціокультурної діяльності,340,3,164.300,0,Українська мова та література (ЗНО): 170\r\nІс...,—
9,2018,м. Київ,Київський університет імені Бориса Грінченка,035,Інститут філології,550,1,167.700,0,Українська мова та література (ЗНО): 178\r\nІн...,—
13,2019,м. Київ,Київський національний університет імені Тарас...,113,Факультет комп`ютерних наук та кібернетики,228,3,190.000,0,Українська мова та література (ЗНО): 185\r\nМа...,—
15,2018,м. Київ,Київський національний університет культури і ...,027,Факультет соціокультурної діяльності,180,2,151.500,0,Українська мова та література (ЗНО): 158\r\nІс...,—
23,2018,м. Київ,Київський національний торговельно-економічний...,035,Факультет торгівлі та маркетингу,636,К,159.018,0,Іноземна мова (ЗНО): 116\r\nІсторія України (З...,—
35,2018,м. Київ,Київський національний університет імені Тарас...,162,ННЦ Інститут біології та медицини,75,6,186.850,0,Українська мова та література (ЗНО): 193\r\nХі...,—
37,2019,м. Київ,Національний університет біоресурсів і природо...,053,Гуманітарно-педагогічний факультет,170,2,149.022,0,Українська мова та література (ЗНО): 186\r\nМа...,—
44,2019,м. Київ,Національний педагогічний університет імені М....,016,Факультет спеціальної та інклюзивної освіти,481,К,120.600,0,Українська мова та література (ЗНО): 122\r\nБі...,—
56,2018,м. Київ,Національний університет «Києво-Могилянська ак...,051,Факультет економічних наук,163,3,186.400,0,Середній бал документа про освіту: 11.2\r\nУкр...,—


In [6]:
budget = df.query("status == 2")
contract = df.query("status == 1").sample(n=5000)
not_passed = df.query("status == 0").sample(n=2000)
df = pd.concat([budget, contract, not_passed])
df

,year,district,university,specialty,faculty,rating,priority,points,status,info,quotas
126,2019,м. Київ,Національна академія образотворчого мистецтва ...,191,Архітектури,10,1,181.500,2,Українська мова та література (ЗНО): 187\r\nМа...,—
183,2018,м. Київ,Національний університет харчових технологій,181,Навчально-науковий інститут харчових технологій,38,—,170.000,2,Харчові технології: 170,—
198,2019,м. Київ,Національний технічний університет України «Ки...,131,Механіко-машинобудівний інститут,16,1,182.172,2,—,—
296,2019,м. Київ,Київський національний університет культури і ...,061,Факультет зв'язків з громадськістю і журналістики,36,4,186.125,2,Українська мова та література (ЗНО): 196\r\nІс...,—
370,2019,м. Київ,Київський національний університет імені Тарас...,101,ННЦ Інститут біології та медицини,7,2,173.655,2,Українська мова та література (ЗНО): 173\r\nБі...,—
433,2018,м. Київ,Київський національний університет будівництва...,192,-,102,4,144.700,2,Українська мова та література (ЗНО): 168\r\nМа...,—
584,2018,м. Київ,Київський національний університет технологій ...,133,Мехатроніки та комп'ютерних технологій,8,—,165.500,2,Середній бал документа про освіту: 75.5\r\nФах...,—
632,2019,м. Київ,Київський національний університет культури і ...,034,Факультет івент-менеджменту та шоу-бізнесу,27,5,181.000,2,Українська мова та література (ЗНО): 194\r\nІс...,—
636,2019,м. Київ,Київський національний університет імені Тарас...,053,Факультет психології,4,1,196.500,2,Українська мова та література (ЗНО): 195\r\nМа...,—
639,2018,м. Київ,Національний технічний університет України «Ки...,131,Механіко-машинобудівний інститут,176,1,130.509,2,Українська мова та література (ЗНО): 141\r\nМа...,—


In [7]:
df = df.drop(["year", "university", "faculty", "info", "quotas", "rating", "priority"], axis=1)
df.status = df.status.dropna()
df.head()
df

,district,specialty,points,status
126,м. Київ,191,181.500,2
183,м. Київ,181,170.000,2
198,м. Київ,131,182.172,2
296,м. Київ,061,186.125,2
370,м. Київ,101,173.655,2
433,м. Київ,192,144.700,2
584,м. Київ,133,165.500,2
632,м. Київ,034,181.000,2
636,м. Київ,053,196.500,2
639,м. Київ,131,130.509,2


## Get values for frontend

In [8]:
df.district.unique()

array(['м. Київ', 'Львівська область', 'Миколаївська область',
       'Запорізька область', 'Харківська область', 'Вінницька область',
       'Рівненська область', 'Одеська область', 'Кіровоградська область',
       'Херсонська область', 'Чернівецька область',
       'Чернігівська область', 'Івано-Франківська область',
       'Дніпропетровська область', 'Черкаська область',
       'Хмельницька область', 'Луганська область',
       'Тернопільська область', 'Сумська область', 'Київська область',
       'Житомирська область', 'Волинська область', 'Донецька область',
       'Полтавська область', 'Закарпатська область'], dtype=object)

In [9]:
df.specialty.unique()

array(['191', '181', '131', '061', '101', '192', '133', '034', '053',
       '201', '162', '125', '035', '072', '172', '152', '123', '124',
       '052', '029', '113', '025', '051', '111', '122', '193', '106',
       '272', '024', '023', '151', '026', '017', '242', '033', '141',
       '202', '173', '032', '231', '207', '121', '016', '014', '161',
       '021', '204', '292', '105', '227', '171', '081', '281', '126',
       '091', '241', '275', '076', '163', '028', '291', '073', '271',
       '142', '075', '022', '134', '208', '015', '103', '182', '132',
       '071', '273', '104', '013', '012', '102', '186', '261', '205',
       '153', '187', '274', '112', '194', '143', '054', '232', '135',
       '223', '144', '183', '145', '136', '293', '185', '206', '226',
       '184', '263', '224', '262', '211', '229', '203', '027', '031',
       '256', '251', '011', '041', '255'], dtype=object)

## Split into two datasets

- The first defines people who didn't pass to contract and didn't pass at all as the same people. This model is used to check people who wants to apply for budget
- The second defines people who pass to contract and to budget as the same category, since if you pass to budget you'll pass to contract

In [10]:
df_budget = df.copy()
df_contract = df.copy()
df_budget["status"] = df_budget["status"].map({
    1: -1,
    2: 1,
    0: -1
})
df_contract["status"] = df_contract["status"].map({
    1: 1,
    2: 1,
    0: -1
})


In [11]:
df_budget.head()

,district,specialty,points,status
126,м. Київ,191,181.500,1
183,м. Київ,181,170.000,1
198,м. Київ,131,182.172,1
296,м. Київ,061,186.125,1
370,м. Київ,101,173.655,1


In [12]:
df_contract.head()

,district,specialty,points,status
126,м. Київ,191,181.500,1
183,м. Київ,181,170.000,1
198,м. Київ,131,182.172,1
296,м. Київ,061,186.125,1
370,м. Київ,101,173.655,1


## Dump cleaned_data set

In [13]:
df_budget.to_csv("budget.csv", index=False)
df_contract.to_csv("contract.csv", index=False)